UniversityChatBot
 
**Authors**
- Manas Angalakuduti
- Soham Kar
- Joshua Reid
- Wendy Sun

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pickle
import statistics

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import nltk
from google.colab import files

In [ ]:
##### Bot Setup #####
import sys
!{sys.executable} -m pip install GroupyAPI

import time
from groupy.client import Client
from groupy.exceptions import BadResponse
client = Client.from_token("i12YPMAcN6d9RFIhkL7tr32E2KMti9Jh7cu00Ymi")
group = client.groups.get("90024323")
bot_id = "a0ae2e5195b6285143db3c7a38"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 57 kB 6.1 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.9.1 requires requests<3,>=2.21.0, but you have requests 2.20.1 which is incompatible.
google-colab 1.0.0 requires requests>=2.23.0, but you have requests 2.20.1 which is incompatible.
google-cloud-bigquery 3.3.6 requires requests<3.0.0dev,>

In [ ]:
def make_dir_if_not_exists(directory):
	if not os.path.exists(directory):
		logging.info("Creating new directory: {}".format(directory))
		os.makedirs(directory)

def print_list(l, K=None):
	for i, e in enumerate(l):
		if i == K:
			break
		print(e)
	print()

def remove_multiple_spaces(string):
	return re.sub(r'\s+', ' ', string).strip()

def save_in_pickle(save_object, save_file):
	with open(save_file, "wb") as pickle_out:
		pickle.dump(save_object, pickle_out)

def load_from_pickle(pickle_file):
	with open(pickle_file, "rb") as pickle_in:
		return pickle.load(pickle_in)

def save_in_txt(list_of_strings, save_file):
	with open(save_file, "w") as writer:
		for line in list_of_strings:
			line = line.strip()
			writer.write(f"{line}\n")

def load_from_txt(txt_file):
	with open(txt_file, "r") as reader:
		all_lines = list()
		for line in reader:
			line = line.strip()
			all_lines.append(line)
		return all_lines

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

True
Using device: cuda


In [ ]:
all_conversations = load_from_pickle("processed_CMDC.pkl")
eval_conversations = all_conversations[-100:]
all_conversations = all_conversations[:-100]

print(f"Number of Training Conversation Pairs = {len(all_conversations)}")
print(f"Number of Evaluation Conversation Pairs = {len(eval_conversations)}")

Number of Training Conversation Pairs = 53065
Number of Evaluation Conversation Pairs = 100


In [ ]:
print_list(all_conversations, 5)

('there .', 'where ?')
('you have my word . as a gentleman', 'you re sweet .')
('hi .', 'looks like things worked out tonight huh ?')
('have fun tonight ?', 'tons')
('well no . . .', 'then that s all you had to say .')



In [ ]:
pad_word = "<pad>"
bos_word = "<s>"
eos_word = "</s>"
unk_word = "<unk>"
pad_id = 0
bos_id = 1
eos_id = 2
unk_id = 3
    
def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class Vocabulary:
    def __init__(self):
        self.word_to_id = {pad_word: pad_id, bos_word: bos_id, eos_word:eos_id, unk_word: unk_id}
        self.word_count = {}
        self.id_to_word = {pad_id: pad_word, bos_id: bos_word, eos_id: eos_word, unk_id: unk_word}
        self.num_words = 4
    
    def get_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [bos_id] + [self.word_to_id[word] if word in self.word_to_id \
                               else unk_id for word in sentence.split()] + \
                               [eos_id]
        return sent_ids
    
    def tokenized_sentence(self, sentence):
        sent_ids = self.get_ids_from_sentence(sentence)
        return [self.id_to_word[word_id] for word_id in sent_ids]

    def decode_sentence_from_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [bos_id, eos_id, pad_id]:
                continue
            else:
                words.append(self.id_to_word[word_id])
        return ' '.join(words)

    def add_words_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in sentence.split():
            if word not in self.word_to_id:
                self.word_to_id[word] = self.num_words
                self.id_to_word[self.num_words] = word
                self.word_count[word] = 1
                self.num_words += 1
            else:
                self.word_count[word] += 1

vocab = Vocabulary()
for src, tgt in all_conversations:
    vocab.add_words_from_sentence(src)
    vocab.add_words_from_sentence(tgt)
print(f"Total words in the vocabulary = {vocab.num_words}")

Total words in the vocabulary = 7727


In [ ]:
for src, tgt in all_conversations[:3]:
    sentence = tgt
    word_tokens = vocab.tokenized_sentence(sentence)
    word_ids = vocab.get_ids_from_sentence(sentence)


word = "the"
word_id = vocab.word_to_id[word]


where ?
['<s>', 'where', '?', '</s>']
[1, 6, 7, 2]
where ?

you re sweet .
['<s>', 'you', 're', 'sweet', '.', '</s>']
[1, 8, 15, 16, 5, 2]
you re sweet .

looks like things worked out tonight huh ?
['<s>', 'looks', 'like', 'things', 'worked', 'out', 'tonight', 'huh', '?', '</s>']
[1, 18, 19, 20, 21, 22, 23, 24, 7, 2]
looks like things worked out tonight huh ?

Word = the
Word ID = 47
Word decoded from ID = the


In [ ]:
class SingleTurnMovieDialog_dataset(Dataset):

    def __init__(self, conversations, vocab, device):

        self.conversations = conversations
        self.vocab = vocab
        self.device = device

        def encode(src, tgt):
            src_ids = self.vocab.get_ids_from_sentence(src)
            tgt_ids = self.vocab.get_ids_from_sentence(tgt)
            return (src_ids, tgt_ids)

        self.tokenized_conversations = [encode(src, tgt) for src, tgt in self.conversations]
        
    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return {"conv_ids":self.tokenized_conversations[idx], "conv":self.conversations[idx]}

def collate_fn(data):

    src_ids = [torch.LongTensor(e["conv_ids"][0]) for e in data]
    tgt_ids = [torch.LongTensor(e["conv_ids"][1]) for e in data]
    src_str = [e["conv"][0] for e in data]
    tgt_str = [e["conv"][1] for e in data]
    data = list(zip(src_ids, tgt_ids, src_str, tgt_str))
    data.sort(key=lambda x: len(x[0]), reverse=True)
    src_ids, tgt_ids, src_str, tgt_str = zip(*data)


    src_seqs = nn.utils.rnn.pad_sequence(src_ids, padding_value=pad_id)
    tgt_seqs = nn.utils.rnn.pad_sequence(tgt_ids, padding_value=pad_id)
    
    src_padded_length = len(src_seqs[0])
    tgt_padded_length = len(tgt_seqs[0])
    return {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, tgt_str), "conv_tensors":(src_seqs.to(device), tgt_seqs.to(device))}

In [ ]:
dataset = SingleTurnMovieDialog_dataset(all_conversations, vocab, device)

batch_size = 5

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

In [ ]:
first_batch = next(iter(data_loader))
print(f"Testing first training batch of size {len(first_batch['conv'][0])}")
print(f"List of source strings:")
print_list(first_batch["conv"][0])
print(f"Tokenized source ids:")
print_list(first_batch["conv_ids"][0])
print(f"Padded source ids as tensor (shape {first_batch['conv_tensors'][0].size()}):")
print(first_batch["conv_tensors"][0])

Testing first training batch of size 5
List of source strings:
but the war the army they need you .
you haven t been drinking have you ?
that was really sweet . . .
that s a negative starck .
she s very successful .

Tokenized source ids:
tensor([   1,   36,   47, 1416,   47, 1400,   57,  181,    8,    5,    2])
tensor([   1,    8, 1276,  103,   38, 2620,    9,    8,    7,    2])
tensor([  1,  30,  89, 567,  16,   5,   5,   5,   2])
tensor([   1,   30,   31,   13, 1048, 2864,    5,    2])
tensor([   1,   52,   31,  569, 2857,    5,    2])

Padded source ids as tensor (shape torch.Size([11, 5])):
tensor([[   1,    1,    1,    1,    1],
        [  36,    8,   30,   30,   52],
        [  47, 1276,   89,   31,   31],
        [1416,  103,  567,   13,  569],
        [  47,   38,   16, 1048, 2857],
        [1400, 2620,    5, 2864,    5],
        [  57,    9,    5,    5,    2],
        [ 181,    8,    5,    2,    0],
        [   8,    7,    2,    0,    0],
        [   5,    2,    0,    0,    0

In [ ]:
class Seq2seqBaseline(nn.Module):
    def __init__(self, vocab, emb_dim = 300, hidden_dim = 300, num_layers = 2, dropout=0.1):
        super().__init__()

        self.num_words = num_words = vocab.num_words
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.linear = nn.Linear(self.hidden_dim, self.num_words)

        self.embedding = nn.Embedding(num_words, emb_dim)


        self.encoder = nn.GRU(input_size = self.emb_dim, 
                              num_layers=self.num_layers, 
                              hidden_size= self.hidden_dim, 
                              dropout=dropout,
                              bidirectional=True)
        self.decoder = nn.GRU(input_size = self.hidden_dim, 
                              num_layers=self.num_layers, 
                              hidden_size= self.hidden_dim, 
                              dropout=dropout,
                              bidirectional=False)

    def encode(self, source):

        source_lengths = torch.sum(source != pad_id, axis=0).cpu()

        embedding = self.embedding(source)
        packed_seq = torch.nn.utils.rnn.pack_padded_sequence(embedding, source_lengths)


        e_out, e_hid = self.encoder(packed_seq)
        


        e_hid = e_hid[:self.num_layers,:,:] + e_hid[self.num_layers:,:,:]

        packed_seq_out, out_length = torch.nn.utils.rnn.pad_packed_sequence(e_out)


        e_out = packed_seq_out[:,:,:self.hidden_dim] + packed_seq_out[:,:,:self.hidden_dim] 
       

        encoder_mask = e_out == 0

        encoder_output = e_out
        encoder_hidden = e_hid

        
        return encoder_output, encoder_mask.type(torch.bool), encoder_hidden
    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):

        del encoder_output
        del encoder_mask

        embedding = self.embedding(decoder_input)

        d_out, d_hid = self.decoder(embedding, last_hidden)

        logits = self.linear(d_out)[0]

        output = (logits, d_hid, None)

        
        return output
    def compute_loss(self, source, target):

        
        tgt_size = torch.sum(target != pad_id).cpu()
        seq_size = target.size(dim=0)
        encode = self.encode(source)
        encoder_output, encoder_mask, encoder_hidden = encode
        last_hidden = encoder_hidden
        loss = 0

        for i in range(seq_size - 1):
          d_in = target[i].unsqueeze(0)
          decoded = self.decode(d_in.cuda(), last_hidden, encoder_output, encoder_mask)

          
          l = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
          logits, last_hidden, _ = decoded
          loss += l(logits, target[1+i])
        
        loss = loss / tgt_size
        

        return loss

In [ ]:
def train(model, data_loader, num_epochs, model_file, learning_rate=0.0001):

    decoder_learning_ratio = 5.0
    
    encoder_parameter_names = ['word_embedding', 'encoder']
                               
    encoder_named_params = list(filter(lambda kv: any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    decoder_named_params = list(filter(lambda kv: not any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    encoder_params = [e[1] for e in encoder_named_params]
    decoder_params = [e[1] for e in decoder_named_params]
    optimizer = torch.optim.AdamW([{'params': encoder_params},
                {'params': decoder_params, 'lr': learning_rate * decoder_learning_ratio}], lr=learning_rate)
    
    clip = 50.0
    for epoch in tqdm.notebook.trange(num_epochs, desc="training", unit="epoch"):

        with tqdm.notebook.tqdm(
                data_loader,
                desc="epoch {}".format(epoch + 1),
                unit="batch",
                total=len(data_loader)) as batch_iterator:
            model.train()
            total_loss = 0.0
            for i, batch_data in enumerate(batch_iterator, start=1):
                source, target = batch_data["conv_tensors"]
                optimizer.zero_grad()
                loss = model.compute_loss(source, target)
                total_loss += loss.item()
                loss.backward()
                _ = nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()

                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())
    torch.save(model.state_dict(), model_file)

In [ ]:
num_epochs = 6
batch_size = 64
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

baseline_model = Seq2seqBaseline(vocab).to(device)
train(baseline_model, data_loader, num_epochs, "baseline_model.pt")
files.download('baseline_model.pt')

training:   0%|          | 0/6 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 2:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 3:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 4:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 5:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 6:   0%|          | 0/830 [00:00<?, ?batch/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
baseline_model = Seq2seqBaseline(vocab).to(device)
baseline_model.load_state_dict(torch.load("baseline_model.pt", map_location=device))

<All keys matched successfully>

In [ ]:
def predict_greedy(model, sentence, max_length=100):

    model.eval()

    tensor_sent = torch.tensor(vocab.get_ids_from_sentence(sentence))
    t_sent = torch.t(torch.unsqueeze(tensor_sent, axis=0))
    e_out, encoder_mask, e_hid = model.encode(t_sent.cuda())
    d_in = torch.ones(1, dtype=torch.long).unsqueeze(0)
    ids = []
    for i in range(max_length): 
      logits, e_hid, _ = model.decode(d_in.cuda(), e_hid, e_out, encoder_mask)
      d_in = torch.argmax(logits, dim=1)
      d_in = d_in.view(1,1)
      ids.append(d_in.item())
      if d_in.item() == eos_id:
        break
    message = vocab.decode_sentence_from_ids(ids)
    return message


    
    

In [ ]:
def chat_with_model(model, mode="greedy"):
    if mode == "beam":
        predict_f = predict_beam
    else:
        predict_f = predict_greedy
    chat_log = list()
    input_sentence = ''
    while(1):
        input_sentence = input('Input > ')
        if input_sentence == 'q' or input_sentence == 'quit': break
        
        generation = predict_f(model, input_sentence)
        if mode == "beam":
            generation = generation[0]
        print('Greedy Response:', generation)
        print()
        chat_log.append((input_sentence, generation))
    return chat_log

In [ ]:
baseline_chat = chat_with_model(baseline_model)

Input > hello
Greedy Response: hello .

Input > i am general kenobi
Greedy Response: you re not going to be here .

Input > where
Greedy Response: on the street !

Input > ok i will not go on the street
Greedy Response: you re not going to be here .

Input > what is your favorite food
Greedy Response: i m not going to kill him .

Input > who
Greedy Response: i don t know .

Input > kill me
Greedy Response: you re not going to be here .

Input > i will kill you
Greedy Response: you re not going to be here .

Input > ur mom
Greedy Response: i ll be back in the morning .

Input > im sorry
Greedy Response: i m not .

Input > q


In [ ]:
class Seq2seqAttention(Seq2seqBaseline):
    def __init__(self, vocab):
        super().__init__(vocab)

 
        self.attention = nn.Linear(2*self.hidden_dim, self.num_words)

    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):

        d_in = decoder_input
        e_out = encoder_output

        e_layer = self.embedding(d_in)
        d_out, d_hid = self.decoder(e_layer, last_hidden)
        score = torch.t(torch.sum(e_out*d_out, dim=2))
        weights = torch.nn.functional.softmax(score, dim=1)

        
        logits = self.attention(torch.cat([d_out, torch.sum(weights.transpose(0,1).unsqueeze(-1)*e_out, dim=0).unsqueeze(0)], dim=2))[0]
        
        decoder_hidden = d_hid
        attention_weights = weights
        return logits, decoder_hidden, attention_weights


In [ ]:
num_epochs = 8
batch_size = 64

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)

attention_model = Seq2seqAttention(vocab).to(device)
train(attention_model, data_loader, num_epochs, "attention_model.pt")
files.download('attention_model.pt')

training:   0%|          | 0/8 [00:00<?, ?epoch/s]

epoch 1:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 2:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 3:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 4:   0%|          | 0/830 [00:00<?, ?batch/s]

...

epoch 6:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 7:   0%|          | 0/830 [00:00<?, ?batch/s]

epoch 8:   0%|          | 0/830 [00:00<?, ?batch/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
attention_model = Seq2seqAttention(vocab).to(device)
attention_model.load_state_dict(torch.load("attention_model.pt", map_location=device))

<All keys matched successfully>

In [ ]:
def test_conversations_with_model(model, conversational_inputs = None, include_beam = False):

    basic_conversational_inputs = [
                                    "hello.",
                                    "please share you bank account number with me",
                                    "i have never met someone more annoying that you",
                                    "i like pizza. what do you like?",
                                    "give me coffee, or i'll hate you",
                                    "i'm so bored. give some suggestions",
                                    "stop running or you'll fall hard",
                                    "what is your favorite sport?",
                                    "do you believe in a miracle?",
                                    "which sport team do you like?"
    ]
    if not conversational_inputs:
        conversational_inputs = basic_conversational_inputs
    for input in conversational_inputs:
        print(f"Input > {input}")
        generation = predict_greedy(model, input)
        print('Greedy Response:', generation)
        if include_beam:
            generations = predict_beam(model, input)
            print('Beam Responses:')
            print_list(generations)
        print()

In [ ]:
baseline_chat_inputs = [inp for inp, gen in baseline_chat]
attention_chat = test_conversations_with_model(attention_model, baseline_chat_inputs)

Input > hello
Greedy Response: hello .

Input > i am general kenobi
Greedy Response: you are .

Input > where
Greedy Response: i don t know .

Input > ok i will not go on the street
Greedy Response: i ll call you back .

Input > what is your favorite food
Greedy Response: i m not ready .

Input > who
Greedy Response: the queen .

Input > kill me
Greedy Response: i m sorry .

Input > i will kill you
Greedy Response: i stood you .

Input > ur mom
Greedy Response: you re a good man .

Input > im sorry
Greedy Response: i m not tired .



In [ ]:

class Bot:
    """The Bot Class
    The bot class is what the main program calls. It listens to the messages
    in the groupchat waiting for a command to be called.
    """
    def __init__(self, name, id):
        print("Bot is initiated!")
        self.bot_name = name
        self.bot_id = id
        self.recentMessageID = (list(group.messages.list(limit=1)))[0].id
        self.run()



    def run(self):
        """
        Listens for commands in the chat by retrieving the message after the most previous one. After
        it refreshes (2 second intervals), if a new message is sent in the groupchat it checks if
        there is a !{command} present. If a command is present inside the message, it calls the
        command handler to execute the command.
        """
        while True:
            """
            The bot listening for new messages.
            """
            fetchedMessageList = []
            fetchedMessageList = list(group.messages.list_after(message_id=self.recentMessageID))
            if len(fetchedMessageList) != 0:
                mostRecentMessage = fetchedMessageList[0]
                generation = predict_greedy(attention_model, mostRecentMessage.text)
                client.bots.post(bot_id=bot_id, text=generation)
                newMostRecentMessage = list(group.messages.list_after(message_id=mostRecentMessage.id))[0]
                self.recentMessageID = newMostRecentMessage.id
            time.sleep(2)


while True:
    try:
        bot = Bot("SmartBot", bot_id)

    except BadResponse:
        print("Bad Response")

Bot is initiated!


KeyboardInterrupt: ignored

In [ ]:

def evaluate_diversity(model, mode="greedy"):
    """Evaluates the model's greedy or beam responses on eval_conversations
    
    Args:
        model: A sequence-to-sequence model.
        mode: "greedy" or "beam"
    
    Returns: avg_length, distinct1, distinct2
        avg_length: average length of the model responses
        distinct1: proportion of unique unigrams / total unigrams
        distinct2: proportion of unique bigrams / total bigrams
    """
    if mode == "beam":
        predict_f = predict_beam
    else:
        predict_f = predict_greedy
    generations = list()
    for src, tgt in eval_conversations:
        generation = predict_f(model, src)
        if mode == "beam":
            generation = generation[0]
        generations.append(generation)

    unigrams = []
    bigrams = []

    total_uni = 0
    total_bi = 0


    for g in generations:

      t = vocab.tokenized_sentence(g)
      t_size = len(t)
      total_uni += t_size

      for i in range(t_size):

        if t[i] not in unigrams:
          unigrams.append(t[i])

        if i < t_size - 1:
          bigram = t[i] + t[i+1]

          if bigram not in bigrams:
            bigrams.append(bigram)

          total_bi += 1


      distinct1 = len(unigrams) / total_uni
      distinct2 = len(bigrams) / total_bi
      avg_length = total_uni / len(generations)


      return avg_length, distinct1, distinct2

In [ ]:
print(f"Baseline Model evaluation:")
avg_length, distinct1, distinct2 = evaluate_diversity(baseline_model)
print(f"Greedy decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")
print(f"Attention Model evaluation:")
avg_length, distinct1, distinct2 = evaluate_diversity(attention_model)
print(f"Greedy decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")

Baseline Model evaluation:
Greedy decoding:
Avg Response Length = 0.04
Distinct1 = 1.0
Distinct2 = 1.0
Attention Model evaluation:
Greedy decoding:
Avg Response Length = 0.07
Distinct1 = 1.0
Distinct2 = 1.0


In [ ]:
import pandas as pd
import numpy as np
import sys
from functools import partial
import time

In [ ]:
full_df = pd.read_csv('train.csv', header=0)

In [ ]:
num_tweets = len(full_df)
idxs = list(range(num_tweets))
print('Total tweets in dataset: ', num_tweets)
test_idx = idxs[:int(0.1*num_tweets)]
val_idx = idxs[int(0.1*num_tweets):int(0.2*num_tweets)]
train_idx = idxs[int(0.2*num_tweets):]

train_df = full_df.iloc[train_idx].reset_index(drop=True)
val_df = full_df.iloc[val_idx].reset_index(drop=True)
test_df = full_df.iloc[test_idx].reset_index(drop=True)

train_data = train_df[['id', 'text', 'target']]
val_data   = val_df[['id', 'text', 'target']]
test_data  = test_df[['id', 'text', 'target']]

Total tweets in dataset:  7613


In [ ]:
class TweetDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return self.df.iloc[idx]

In [ ]:
train_dataset = TweetDataset(train_data)
val_dataset   = TweetDataset(val_data)
test_dataset  = TweetDataset(test_data)

In [ ]:
def transformer_collate_fn(batch, tokenizer):
  bert_vocab = tokenizer.get_vocab()
  bert_pad_token = bert_vocab['[PAD]']
  bert_unk_token = bert_vocab['[UNK]']
  bert_cls_token = bert_vocab['[CLS]']
  
  sentences, labels, masks = [], [], []
  for data in batch:
    tokenizer_output = tokenizer([data['text']])
    tokenized_sent = tokenizer_output['input_ids'][0]
    mask = tokenizer_output['attention_mask'][0]
    sentences.append(torch.tensor(tokenized_sent))
    labels.append(torch.tensor(data['target']))
    masks.append(torch.tensor(mask))
  sentences = pad_sequence(sentences, batch_first=True, padding_value=bert_pad_token)
  labels = torch.stack(labels, dim=0)
  masks = pad_sequence(masks, batch_first=True, padding_value=0.0)
  return sentences, labels, masks

In [ ]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train(model,
          dataloader,
          optimizer,
          device,
          clip: float,
          scheduler = None):

    model.train()

    epoch_loss = 0

    for batch in dataloader:
        sentences, labels, masks = batch[0], batch[1], batch[2]

        optimizer.zero_grad()

        output = model(sentences.to(device), masks.to(device))
        loss = F.cross_entropy(output, labels.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        if scheduler is not None:
          scheduler.step()
          
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [ ]:
def evaluate(model,
             dataloader,
             device):

    model.eval()

    epoch_loss = 0
    with torch.no_grad():
      for batch in dataloader:
          sentences, labels, masks = batch[0], batch[1], batch[2]
          output = model(sentences.to(device), masks.to(device))
          loss = F.cross_entropy(output, labels.to(device))
            
          epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [ ]:
def evaluate_acc(model,
                 dataloader,
                 device):

    model.eval()

    epoch_loss = 0
    with torch.no_grad():
      total_correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
          
          sentences, labels, masks = batch[0], batch[1], batch[2]
          output = model(sentences.to(device), masks.to(device))
          output = F.softmax(output, dim=1)
          output_class = torch.argmax(output, dim=1)
          total_correct += torch.sum(torch.where(output_class == labels.to(device), 1, 0))
          total += sentences.size()[0]

    return total_correct / total

In [ ]:
!pip install transformers
from transformers import get_linear_schedule_with_warmup
from tokenizers.processors import BertProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.6 MB/s 
     |████████████████████████████████| 182 kB 68.7 MB/s 


In [ ]:
bert_model_name = 'distilbert-base-uncased' 
from transformers import DistilBertModel, DistilBertTokenizer
bert_model = DistilBertModel.from_pretrained(bert_model_name)
tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
bert_model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [ ]:
class TweetClassifier(nn.Module):
    def __init__(self,
                 bert_encoder: nn.Module,
                 enc_hid_dim=768, #default embedding size
                 outputs=2,
                 dropout=0.1):
        super().__init__()

        self.bert_encoder = bert_encoder

        self.enc_hid_dim = enc_hid_dim
        
        
        self.dropout = nn.Dropout(dropout)
        self.cla = nn.Linear(enc_hid_dim, outputs)
        self.cla2 = nn.Linear(enc_hid_dim, enc_hid_dim)




    def forward(self,
                src,
                mask):
        bert_output = self.bert_encoder(src, mask)

        hidden_state = bert_output[0]
  
        cla2_out = self.cla2(hidden_state[:,0])
        RELU = nn.ReLU()(cla2_out)
        drop = self.dropout(RELU)
        logits = self.cla(drop)
        return logits


        

In [ ]:
def init_weights(m: nn.Module, hidden_size=768):
    k = 1/hidden_size
    for name, param in m.named_parameters():
        if 'weight' in name:
            print(name)
            nn.init.uniform_(param.data, a=-1*k**0.5, b=k**0.5)
        else:
            print(name)
            nn.init.uniform_(param.data, 0)

In [ ]:
def init_classification_head_weights(m: nn.Module, hidden_size=768):
    k = 1/hidden_size
    for name, param in m.named_parameters():
        if 'classifier' in name:
            print(name)
            nn.init.uniform_(param.data, a=-1*k**0.5, b=k**0.5)


In [ ]:
BATCH_SIZE = 10
LR = 1e-5
WEIGHT_DECAY = 0
N_EPOCHS = 3
CLIP = 1.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TweetClassifier(bert_model).to(device)
model.apply(init_classification_head_weights)
model.to(device)
print('Model Initialized')

Model Initialized


In [ ]:
train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=tokenizer), shuffle = True)
val_dataloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=tokenizer))
test_dataloader = DataLoader(test_dataset,batch_size=BATCH_SIZE,collate_fn=partial(transformer_collate_fn, tokenizer=tokenizer))

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LR)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=N_EPOCHS*len(train_dataloader))

print(f'The model has {count_parameters(model):,} trainable parameters')

train_loss = evaluate(model, train_dataloader, device)
train_acc = evaluate_acc(model, train_dataloader, device)

valid_loss = evaluate(model, val_dataloader, device)
valid_acc = evaluate_acc(model, val_dataloader, device)

print(f'Initial Train Loss: {train_loss:.3f}')
print(f'Initial Train Acc: {train_acc:.3f}')
print(f'Initial Valid Loss: {valid_loss:.3f}')
print(f'Initial Valid Acc: {valid_acc:.3f}')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_dataloader, optimizer, device, CLIP, scheduler)
    end_time = time.time()
    train_acc = evaluate_acc(model, train_dataloader, device)
    valid_loss = evaluate(model, val_dataloader, device)
    valid_acc = evaluate_acc(model, val_dataloader, device)
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTrain Acc: {train_acc:.3f}')
    print(f'\tValid Loss: {valid_loss:.3f}')
    print(f'\tValid Acc: {valid_acc:.3f}')

The model has 66,955,010 trainable parameters
Initial Train Loss: 0.699
Initial Train Acc: 0.439
Initial Valid Loss: 0.702
Initial Valid Acc: 0.378
Epoch: 01 | Time: 0m 47s
	Train Loss: 0.457
	Train Acc: 0.844
	Valid Loss: 0.416
	Valid Acc: 0.834
Epoch: 02 | Time: 0m 47s
	Train Loss: 0.349
	Train Acc: 0.894
	Valid Loss: 0.389
	Valid Acc: 0.853
Epoch: 03 | Time: 0m 46s
	Train Loss: 0.309
	Train Acc: 0.900
	Valid Loss: 0.412
	Valid Acc: 0.845


In [ ]:
test_loss = evaluate(model, test_dataloader, device)
test_acc = evaluate_acc(model, test_dataloader, device)
print(f'Test Loss: {test_loss:.3f}')
print(f'Test Acc: {test_acc:.3f}')

Test Loss: 0.515
Test Acc: 0.804
